# Критерии остановки

*Общие закономерности при наборе данных*  
*Критерии остановки*  
*Выбор порогов*  
*Оценка длительности*  
*Минимальная длительность*

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.graph_objects as go

np.random.seed(7)

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
plt.rcParams.update({
    "figure.facecolor": "white",
    "axes.facecolor": "white",
    "axes.grid": True,
    "grid.color": "whitesmoke",
    "axes.axisbelow": True,
    "axes.spines.top": False,
    "axes.spines.right": False,
    "axes.titlesize": 14,
    "axes.titlepad": 20,
    "axes.titlelocation": "left",
    "lines.linewidth": 1.3,
    "legend.frameon": False,
    "figure.figsize": (10, 5),
    "figure.dpi": 200,
    "savefig.dpi": 200,
})
np.random.seed(7)

# Общие закономерности

Оценка эффекта уточняется, дисперсия снижается.  
Для конверсий и средних стандартное отклонение разности снижается $\sim 1/\sqrt{N}$.

-Если между группами небольшая разница, не интересно дожидаться вероятности 95%.  
Если эффект меньше заданной величины, эксперимент пора прекращать.

-Нужно учитывать последствия. Пороговые значения $c$ и $p$ должны учитывать последствия.
  
-Продолжение эксперимента - потери. 

-Ценность данных снижается: ценность больше, когда данных мало. Меньше, когда данных уже много.

-Цена продолжения эксп. может нарастать (больше человек в неоптимальной группе). 

# Критерии остановки

Варианты:
- Метрика одной группы лучше другой $ P(p_B > p_A) > c \cup P(p_A > p_B) > c $
- Нет эффекта: $P(|p_B - p_A| < \delta) > c$
- Нет ухудшения: $P(p_B - p_A < -\delta) < c$

С учетом эффекта:  
- $E[Gain] > k E[Loss]$
- $P(Loss > \delta) < c$, $P(Gain < \delta) > c$
- $E[Loss] < L$, $E[\delta] > 0$


Еще
- Снижение риска $<$ стоимости данных: $E[Loss(N)] - E[Loss(N + dN)] < Cost(dN)$


Нужно:  
-выбрать критерии  
-определить пороги    
-оценить длительность  
-оценить минимальную длительность  

Как выбрать пороги.  
Пороги - это соотношение количество релизов-точность.   
Больше точность - дольше эксперименты.  

Стоит целиться в значимые улучшения.  
Сильный эффект. Видно быстро.  
Точность менее важна.  
Нужен контроль ошибок.  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import matplotlib.ticker as mticker

d = stats.norm(loc=0.3, scale=0.5)
delta = 0.9
x = np.linspace(-5, 5, 1000)
y = d.pdf(x)
ymax = y.max()

fig, axes = plt.subplots(1, 3, figsize=(15, 4), sharey=True)

# P(B > A) > c
ax = axes[0]
ax.plot(x, y, color='black')
ax.axvline(0, linestyle='dashed', color='black')
ax.fill_between(x, y, where=(x > 0), alpha=0.3, color='black')
ax.text(0, 1.15 * y.max(), '0', ha='center', va='top')
ax.set_title(r'$P(B > A) > c$')
ax.set_ylabel('Probability Density')

# P(|B - A| < δ) > c
ax = axes[1]
ax.plot(x, y, color='black')
ax.axvline(-delta, linestyle='dashed', color='black')
ax.axvline(delta, linestyle='dashed', color='black')
ax.fill_between(x, y, where=(np.abs(x) < delta), alpha=0.3, color='black')
ax.text(-delta, 1.15 * ymax, r'$-\delta$', ha='center', va='top')
ax.text(delta, 1.15 * ymax, r'$+\delta$', ha='center', va='top')
ax.set_title(r'$P(|B - A| < \delta) > c$')

# P(B - A < -δ) < c
ax = axes[2]
ax.plot(x, y, color='black')
ax.axvline(-delta, linestyle='dashed', color='black')
ax.fill_between(x, y, where=(x < -delta), alpha=0.3, color='black')
ax.text(-delta, 1.15 * y.max(), r'$-\delta$', ha='center', va='top')
ax.set_title(r'$P(B - A < -\delta) < c$')

for ax in axes:
    ax.xaxis.set_major_formatter(mticker.FormatStrFormatter('%.1f'))

plt.tight_layout()
plt.show()


In [ ]:
# Posterior for B - A
mu = 0.5
sigma = 1.5
d = stats.norm(loc=mu, scale=sigma)

x = np.linspace(-5, 5, 1000)
pdf = d.pdf(x)
ymax = pdf.max()


# Business / stopping parameters
k = 2.0
L = 0.2
delta = 0.1
c = 0.95

# --- Expected values via truncated normal using d.cdf ---
a_pos = (0 - mu) / sigma
b_neg = (0 - mu) / sigma

# Positive part: X | X > 0
tn_pos = stats.truncnorm(a_pos, np.inf, loc=mu, scale=sigma)
P_pos = 1 - d.cdf(0)
E_pos = P_pos * tn_pos.mean()  # expectation conditional X>0

# Negative part: X | X < 0
tn_neg = stats.truncnorm(-np.inf, b_neg, loc=mu, scale=sigma)
P_neg = d.cdf(0)
E_neg = P_neg * tn_neg.mean()  # expectation conditional X<0

fig, axes = plt.subplots(1, 3, figsize=(15, 4), sharey=False)

# E[Gain] > k E[Loss]
ax = axes[0]
ax.plot(x, pdf, color='black')
ax.axvline(0, linestyle='solid', color='black')
ax.axvline(E_pos, linestyle='dashed', color='black')
ax.axvline(E_neg, linestyle='dashed', color='gray')
ax.text(E_pos, 1.07*ymax, '  E[B>0]', ha='left', va='top')
ax.text(E_neg, 1.07*ymax, 'E[B<0]  ', ha='right', va='top')
#ax.text(0, 1.15*ymax, '0', ha='center', va='top')
ax.set_title(r'$E[B-A>0] > k\,E[B-A<0]$')
ax.set_ylabel('Probability Density')

# P(Gain < δ) < c
ax = axes[1]
ax.plot(x, pdf, color='black')
ax.axvline(delta, linestyle='dashed', color='black')
ax.axvline(E_pos, linestyle='dashed', color='black')
ax.axvline(0, linestyle='solid', color='black')
ax.text(delta, 1.15*ymax, r'$\delta$', ha='center', va='top')
ax.text(E_pos, 1.10*ymax, 'E[B>0]', ha='left', va='top')
ax.set_title(r'$P(B-A < \delta) < c$')
ax.set_ylabel('Probability Density')

# E[Loss] < L
ax = axes[2]
ax.plot(x, pdf, color='black')
ax.axvline(0, linestyle='solid', color='black')
ax.axvline(E_neg, linestyle='dashed', color='black')
ax.text(E_neg, 1.15*ymax, 'E[B<0]', ha='right', va='top')
#ax.text(0, 1.15*ymax, '0', ha='center', va='top')
ax.set_title(r'$E[B-A<0] < L$')
ax.set_ylabel('Probability Density')

for ax in axes:
    ax.xaxis.set_major_formatter(mticker.FormatStrFormatter('%.1f'))

plt.tight_layout()
plt.show()


# Выбор порогов

# Оценка длительности

Notation:
$$
\begin{aligned}
& \Delta = B-A \quad \text{(true lift)} \\
& \hat\mu = \text{posterior mean of } \Delta \\
& \hat\sigma^2 = \text{posterior variance of } \Delta \\
& p = \text{Bernoulli probability of success} \\
& N = \text{number of observations per group} \\
& c = \text{stopping probability threshold} \\
& \delta = \text{equivalence / minimum gain threshold} \\
& k = \text{gain/loss ratio threshold} \\
& L = \text{maximum expected loss allowed}
\end{aligned}
$$

Probability threshold: $P(B>A) > c$
$$
P(\Delta > 0) = 1 - \Phi\left(\frac{0 - \hat\mu}{\hat\sigma}\right) = c
\quad \Rightarrow \quad
\hat\sigma = \frac{\hat\mu}{\Phi^{-1}(c)}
\quad \Rightarrow \quad
N \approx \frac{p(1-p)}{(\hat\mu / \Phi^{-1}(c))^2}
$$

No-harm threshold: $P(B-A < -\delta) < c$
$$
\\
P(\Delta < -\delta) = \Phi\left(\frac{-\delta - \hat\mu}{\hat\sigma}\right) < c
\quad \Rightarrow \quad
\hat\sigma = \frac{\delta + \hat\mu}{\Phi^{-1}(c)}
\quad \Rightarrow \quad
N \approx \frac{p(1-p)}{((\delta+\hat\mu)/\Phi^{-1}(c))^2}
$$

No-effect / equivalence: $P(|B-A| < \delta) > c$
$$
P(|\Delta| < \delta) \approx 2\Phi\left(\frac{\delta}{\hat\sigma}\right)-1 = c
\quad \Rightarrow \quad
\hat\sigma = \frac{\delta}{\Phi^{-1}((c+1)/2)}
\quad \Rightarrow \quad
N \approx \frac{p(1-p)}{(\delta / \Phi^{-1}((c+1)/2))^2}
$$

Utility-based: $E[\text{Gain}] > k E[\text{Loss}]$
$$
\text{Ship if } E[\text{Gain}] - k E[\text{Loss}] > 0
\quad \Rightarrow \quad
N \approx \frac{\mathrm{Var}(\text{Gain}-k\text{Loss})}{(E[\text{Gain}]-k E[\text{Loss}])^2}
$$

Utility-based: $E[\text{Loss}] < L$
$$
\\
\text{Ship if } E[\text{Loss}] < L
\quad \Rightarrow \quad
N \approx \frac{\mathrm{Var}(\text{Loss})}{(L - E[\text{Loss}])^2}
$$

Minimum gain: $P(\text{Gain} < \delta) < c$
$$
P(\Delta < \delta) = \Phi\left(\frac{\delta - \hat\mu}{\hat\sigma}\right) < c
\quad \Rightarrow \quad
\hat\sigma = \frac{\hat\mu - \delta}{\Phi^{-1}(c)}
\quad \Rightarrow \quad
N \approx \frac{\hat\sigma_0^2 (\Phi^{-1}(c))^2}{(\hat\mu-\delta)^2}
$$

$$
\text{Summary Table of Required Sample Sizes:}
\\
\begin{array}{|l|l|}
\hline
\text{Stopping Condition} & N \text{ (observations per group)} \\
\hline
P(B>A) > c & \frac{p(1-p)}{(\hat\mu / \Phi^{-1}(c))^2} \\
P(B-A < -\delta) < c & \frac{p(1-p)}{((\delta+\hat\mu)/\Phi^{-1}(c))^2} \\
P(|B-A|<\delta) > c & \frac{p(1-p)}{(\delta / \Phi^{-1}((c+1)/2))^2} \\
E[\text{Gain}] > k E[\text{Loss}] & \frac{\mathrm{Var}(\text{Gain}-k\text{Loss})}{(E[\text{Gain}]-k E[\text{Loss}])^2} \\
E[\text{Loss}] < L & \frac{\mathrm{Var}(\text{Loss})}{(L - E[\text{Loss}])^2} \\
P(\text{Gain}<\delta)<c & \frac{\hat\sigma^2 (\Phi^{-1}(c))^2}{(\hat\mu-\delta)^2} \\
\hline
\end{array}
$$


# Минимальная длительность

$$
\frac{p_B - p_A}{p_A} < 0.03
$$

Относительная разность меньше 3% с вероятностью 95% .
$$
P\left(\frac{p_B - p_A}{p_A} < 0.03\right) > 95 \%
$$

$$
P\left(\frac{p_B}{p_A} < 1.03\right) > 95 \%
$$

Нужно учесть знак.

$$
\frac{|p_B - p_A|}{p_A} < 0.03
$$

Начать с абсолютной разности.

$$
P(|p_B - p_A| < c)
\\
P_{p_B - p_A}(x) 
\approx \mbox{Norm}\left(x; \mu_B - \mu_A, \sigma_A^2 + \sigma_B^2\right)
\\
\\
\begin{split}
P(|p_B - p_A| < c) & = P(p_B - p_A < c) + P(p_B - p_A > 1 - c) 
\\
& = F_{p_B - p_A}(c) + 1 - F_{p_B - p_A}(1 - c)
\\
& = 1 - 2 F_{p_B - p_A}(c)
\end{split}
$$

Условие остановки: либо одна группа лучше другой $P(B>A) > 95\%$, либо разница несущественна $P(|B-A| < c) > 95\%$.

То же для относительной разности.

$$
P(\frac{|p_B - p_A|}{p_A} < c)
$$

Следующий вопрос - как выбрать $c$ и $95\%$?  
Здесь понадобится оценка последствий.  

Вероятность одной группы лучше другой связана с оценкой эффекта?

Нарисовать картинку с динамикой.  
Разность по мере набора данных.  
Полоса [-c, c].

In [ ]:
c = 2.5
x= np.linspace(-7, 7, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x),
                        line_color='black'))
fig.add_trace(go.Scatter(x=[c, c], y=[0, max(stats.norm.pdf(x)*1.1)],
                         mode='lines', line_dash='dash', line_color='black'))
fig.add_trace(go.Scatter(x=[-c, -c], y=[0, max(stats.norm.pdf(x)*1.1)],
                         mode='lines', line_dash='dash', line_color='black'))
fig.update_layout(
    height=500,
    template='plotly_white')
fig.show()

In [ ]:
def posterior_dist_binom(ns, ntotal, a_prior=1, b_prior=1):
    a = a_prior + ns
    b = b_prior + ntotal - ns 
    return stats.beta(a=a, b=b)

def prob_pb_gt_pa(post_dist_A, post_dist_B, post_samp=100_000):
    sa = post_dist_A.rvs(size=post_samp)
    sb = post_dist_B.rvs(size=post_samp)
    b_gt_a = np.sum(sb > sa)
    return b_gt_a / post_samp

def posterior_binom_approx_95pdi(post_dist):
    lower = post_dist.ppf(0.025)
    upper = post_dist.ppf(0.975)
    return lower, upper

pa = 0.1
pb = pa * 1.01

npoints = 1000
nstep = 150
sa = stats.binom.rvs(p=pa, n=npoints, size=nstep)
sb = stats.binom.rvs(p=pb, n=npoints, size=nstep)

df = pd.DataFrame()
df['npoints'] = [npoints] * nstep
df['sa_step'] = sa
df['sb_step'] = sb
df['N'] = df['npoints'].cumsum()
df['sa'] = df['sa_step'].cumsum()
df['sb'] = df['sb_step'].cumsum()
df['pa'] = df.apply(lambda r: posterior_dist_binom(r['sa'], r['N']).mean(), axis=1)
df[['pa_lower', 'pa_upper']] = df.apply(lambda r: posterior_binom_approx_95pdi(posterior_dist_binom(r['sa'], r['N'])), axis=1, result_type="expand")
df['pb'] = df.apply(lambda r: posterior_dist_binom(r['sb'], r['N']).mean(), axis=1)
df[['pb_lower', 'pb_upper']] = df.apply(lambda r: posterior_binom_approx_95pdi(posterior_dist_binom(r['sb'], r['N'])), axis=1, result_type="expand")
df['pb_gt_pa'] = df.apply(lambda r: prob_pb_gt_pa(posterior_dist_binom(r['sa'], r['N']), posterior_dist_binom(r['sb'], r['N']), post_samp=10_000), axis=1)
df['diff_mu'] = df.apply(lambda r: r['pb'] - r['pa'], axis=1)
df['diff_s'] = df.apply(lambda r: np.sqrt(posterior_dist_binom(r['sa'], r['N']).std()**2 + posterior_dist_binom(r['sb'], r['N']).std()**2), axis=1)
df[['diff_lower', 'diff_upper']] = df.apply(lambda r: (r['diff_mu'] - 2*r['diff_s'], r['diff_mu'] + 2*r['diff_s']), axis=1, result_type="expand")

# xaxis_min = -0.1
# xaxis_max = 0.1  
# x = np.linspace(xaxis_min, xaxis_max, 1000)
# fig = go.Figure()
# for N, o in [(1000, 0.3), (10000, 1)]:
#     dist_a = posterior_dist_binom(df[df['N'] == N]['sa'], N)
#     dist_b = posterior_dist_binom(df[df['N'] == N]['sb'], N)
#     #fig.add_trace(go.Scatter(x=x, y=dist_a.pdf(x), line_color='red', opacity=o, name=f'А, N={N}'))
#     #fig.add_trace(go.Scatter(x=x, y=dist_b.pdf(x), line_color='blue', opacity=o, name=f'Б, N={N}'))
#     #
#     diff = stats.norm(loc=df[df['N'] == N]['diff_mu'], scale=df[df['N'] == N]['diff_s'])
#     fig.add_trace(go.Scatter(x=x, y=diff.pdf(x), line_color='black', opacity=o, name=f'Diff, N={N}'))
#     fig.update_layout(title='$p_B - p_A$',
#                       xaxis_title='$p$',
#                       yaxis_title='Плотность вероятности',
#                       xaxis_range=[xaxis_min, xaxis_max],
#                       hovermode="x",
#                       height=500)
# fig.show()


c = 0.01
fig = go.Figure()
# fig.add_trace(go.Scatter(x=df['N'], y=df['pa'], name='$p_A$',
#                          line_color='red'))
# fig.add_trace(go.Scatter(x=list(df['N']) + list(reversed(df['N'])), 
#                          y=list(df['pa_upper']) + list(reversed(df['pa_lower'])),
#                          fill="toself", name='$p_A, \, \mbox{95% PDI}$', marker_color='red', opacity=0.2))
# fig.add_trace(go.Scatter(x=df['N'], y=df['pb'], name='$p_B$',
#                          line_color='blue'))
# fig.add_trace(go.Scatter(x=list(df['N']) + list(reversed(df['N'])), 
#                          y=list(df['pb_upper']) + list(reversed(df['pb_lower'])),
#                          fill="toself", name='$p_B, \, \mbox{95% PDI}$', marker_color='blue', opacity=0.2))
fig.add_trace(go.Scatter(x=df['N'], y=df['diff_mu'], name='$p_B-p_A$',
                         line_color='black'))
fig.add_trace(go.Scatter(x=list(df['N']) + list(reversed(df['N'])), 
                         y=list(df['diff_lower']) + list(reversed(df['diff_upper'])),
                         fill="toself", name='$p_B - p_A, \, \mbox{95% PDI}$', marker_color='black', opacity=0.2))
fig.add_trace(go.Scatter(x=[df['N'][0], df['N'].iloc[-1]], y=[c, c], name='C',
                         line_color='black', line_dash='dash'))
fig.add_trace(go.Scatter(x=[df['N'][0], df['N'].iloc[-1]], y=[-c, -c], name='-C',
                         line_color='black', line_dash='dash'))
fig.update_layout(title='$p_A, p_B$',
                  yaxis_tickformat = ',.1%',
                  xaxis_title='N',
                  height=600)
fig.show()


# fig = go.Figure()
# fig.add_trace(go.Scatter(x=df['N'], y=df['pb_gt_pa'],
#                          line_color='black', name='$p_B > p_A$'))
# fig.update_layout(title='$P(p_B > p_A)$',
#                   xaxis_title='$N$',
#                   #yaxis_title='sigma',
#                   yaxis_range=[0, 1],
#                   #xaxis_range=[-0.1, 0.1],
#                   hovermode="x",
#                   height=500)  
# fig.show()

In [ ]:
d = stats.norm(loc=0.5, scale=0.5)
delta = 0.5


x = np.linspace(-5, 5, 1000)
y = d.pdf(x)
plt.figure()
plt.plot(x, y, color='black', label=r'$B-A$')
plt.plot([0, 0], [0, y.max()], 
          color='black', linestyle='dashed')
plt.fill_between(x, y, where=(x > 0),
    color='black', alpha=0.3, label=r'$P(B - A > 0)$')
plt.title("B - A")
plt.xlabel("p")
plt.ylabel("Probability density")
#plt.xlim(- 0.1, 0.1)
plt.gca().xaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))
plt.legend()
plt.show()



plt.figure()
plt.plot(x, y, color='black', label=r'$B-A$')
plt.axvline(-delta, color='black', linestyle='dashed')

plt.fill_between(
    x, y,
    where=(x < -delta),
    color='black', alpha=0.3,
    label=r'$P(B-A<-\delta)$'
)

plt.title(r'$P(p_B - p_A < -\delta) < c$  (No harm)')
plt.xlabel(r'$B-A$')
plt.ylabel('Probability density')
plt.gca().xaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))
plt.legend()
plt.show()



plt.figure()
plt.plot(x, y, color='black', label=r'$B-A$')
plt.axvline(-delta, color='black', linestyle='dashed')
plt.axvline(delta, color='black', linestyle='dashed')

plt.fill_between(
    x, y,
    where=(np.abs(x) < delta),
    color='black', alpha=0.3,
    label=r'$P(|B-A|<\delta)$'
)

plt.title(r'$P(|p_B - p_A| < \delta) > c$  (No effect)')
plt.xlabel(r'$B-A$')
plt.ylabel('Probability density')
plt.gca().xaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))
plt.legend()
plt.show()


In [ ]:
# Posterior for B - A
mu = 0.5
sigma = 0.5
d = stats.norm(loc=mu, scale=sigma)

x = np.linspace(-5, 5, 1000)
pdf = d.pdf(x)

# Business parameters
g = 1.0     # gain multiplier
l = 1.0     # loss multiplier
k = 2.0     # gain/loss ratio threshold
L = 0.2     # loss budget
delta = 0.1
c = 0.95

# Gain / loss functions
gain = g * np.maximum(x, 0)
loss = l * np.maximum(-x, 0)

fig, axes = plt.subplots(1, 3, figsize=(15, 4), sharey=False)

# ---------- 1) E[Gain] > k E[Loss] ----------
ax = axes[0]
ax.plot(x, pdf, color='black')
ax.axvline(0, linestyle='dashed', color='black')
ax.fill_between(x, pdf * gain, where=(x > 0), alpha=0.3, color='black')
ax.fill_between(x, pdf * loss, where=(x < 0), alpha=0.3, color='gray')
ax.set_title(r'$E[\mathrm{Gain}] > k\,E[\mathrm{Loss}]$')
ax.set_xlabel('B - A')
ax.set_ylabel('Weighted Density')

# ---------- 2) E[Loss] < L ----------
ax = axes[1]
ax.plot(x, pdf, color='black')
ax.axvline(0, linestyle='dashed', color='black')
ax.fill_between(x, pdf * loss, where=(x < 0), alpha=0.3, color='black')
ax.set_title(r'$E[\mathrm{Loss}] < L$')
ax.set_xlabel('B - A')
ax.set_ylabel('Expected Loss Density')

# ---------- 3) P(Gain < δ) < c ----------
ax = axes[2]
ax.plot(x, pdf, color='black')
ax.axvline(delta, linestyle='dashed', color='black')
ax.fill_between(x, pdf, where=(x < delta), alpha=0.3, color='black')
ax.set_title(r'$P(\mathrm{Gain} < \delta) < c$')
ax.set_xlabel('B - A')
ax.set_ylabel('Probability Density')

for ax in axes:
    ax.xaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))

plt.tight_layout()
plt.show()
